<a href="https://colab.research.google.com/github/sominshim/map/blob/main/EDA_%EB%B3%B5%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Visualization HW #4 17011720 심소민

Due date: Dec 02 Wed

data: [Chicago Crimes - 2001 to present](https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present-Dashboard/5cd6-ry5g)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.core.display import display, HTML, Javascript
from string import Template
import pandas as pd
import numpy as np
import json, random
import IPython.display
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn')

import warnings
warnings.filterwarnings('ignore')

In [3]:
filename = '/content/drive/My Drive/Crimes_-_2001_to_Present.csv'
data = pd.read_csv(filename)
data.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,11034701,JA366925,01/01/2001 11:00:00 AM,016XX E 86TH PL,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,412,4.0,8.0,45.0,11,NaN,NaN,2001,08/05/2017 03:50:08 PM,NaN,NaN,NaN
1,11227287,JB147188,10/08/2017 03:00:00 AM,092XX S RACINE AVE,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,RESIDENCE,False,False,2222,22.0,21.0,73.0,02,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
2,11227583,JB147595,03/28/2017 02:00:00 PM,026XX W 79TH ST,0620,BURGLARY,UNLAWFUL ENTRY,OTHER,False,False,835,8.0,18.0,70.0,05,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
3,11227293,JB147230,09/09/2017 08:17:00 PM,060XX S EBERHART AVE,0810,THEFT,OVER $500,RESIDENCE,False,False,313,3.0,20.0,42.0,06,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN
4,11227634,JB147599,08/26/2017 10:00:00 AM,001XX W RANDOLPH ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,HOTEL/MOTEL,False,False,122,1.0,42.0,32.0,02,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN


경범죄와 중범죄 구분 기준
: 어떤 범주로 형벌이 가해지는지 [FindLaw](https://www.findlaw.com/), [FreeAdvice](https://www.freeadvice.com/)등 사이트를 참고하였다. 정도에 따라 달라지는 것은 rough하게 often/general한 경우로 취급하였다. 그 외의 언급이 없는 경우, 경계가 애매하므로 애매함 카테고리로 분류한다.

In [4]:
Felony = ['CRIMINAL SEXUAL ASSAULT', 'BURGLARY', 'SEX OFFENSE', 'CRIM SEXUAL ASSAULT', 'SEX OFFENSE', 'NARCOTICS', 
          'ROBBERY', 'ARSON', 'KIDNAPPING', 'HUMAN TRAFFICKING']
Vagueness = ['OFFENSE INVOLVING CHILDREN', 'MOTOR VEHICLE THEFT', 'GAMBLING', 'STALKING', 'OTHER NARCOTIC VIOLATION', 
             'CONCEALED CARRY LICENSE VIOLATION', 'RITUALISM']
Misdemeanor = ['DECEPTIVE PRACTICE', 'THEFT', 'CRIMINAL DAMAGE', 'OTHER OFFENSE', 'BATTERY', 'ASSAULT', 'CRIMINAL TRESPASS', 
               'WEAPONS VIOLATION', 'OBSCENITY', 'LIQUOR LAW VIOLATION', 'PROSTITUTION', 'NON-CRIMINAL', 
               'PUBLIC PEACE VIOLATION', 'INTIMIDATION', 'INTERFERENCE WITH PUBLIC OFFICER', 'PUBLIC INDECENCY', 
               'NON-CRIMINAL (SUBJECT SPECIFIED)', 'DOMESTIC VIOLENCE']
HOMICIDE = ['HOMICIDE']

In [5]:
len(Felony)+len(Vagueness)+len(Misdemeanor)+len(HOMICIDE)==len(data['Primary Type'].unique())

True

In [6]:
data['Felony'] = 0
data.loc[data['Primary Type'].isin(Felony),'Felony'] = 1
data['Vagueness'] = 0
data.loc[data['Primary Type'].isin(Vagueness),'Vagueness'] = 1
data['Misdemeanor'] = 0
data.loc[data['Primary Type'].isin(Misdemeanor),'Misdemeanor'] = 1
data['HOMICIDE'] = 0
data.loc[data['Primary Type'].isin(HOMICIDE),'HOMICIDE'] = 1
data.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Felony,Vagueness,Misdemeanor,HOMICIDE
0,11034701,JA366925,01/01/2001 11:00:00 AM,016XX E 86TH PL,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,412,4.0,8.0,45.0,11,NaN,NaN,2001,08/05/2017 03:50:08 PM,NaN,NaN,NaN,0,0,1,0
1,11227287,JB147188,10/08/2017 03:00:00 AM,092XX S RACINE AVE,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,RESIDENCE,False,False,2222,22.0,21.0,73.0,02,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN,1,0,0,0
2,11227583,JB147595,03/28/2017 02:00:00 PM,026XX W 79TH ST,0620,BURGLARY,UNLAWFUL ENTRY,OTHER,False,False,835,8.0,18.0,70.0,05,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN,1,0,0,0
3,11227293,JB147230,09/09/2017 08:17:00 PM,060XX S EBERHART AVE,0810,THEFT,OVER $500,RESIDENCE,False,False,313,3.0,20.0,42.0,06,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN,0,0,1,0
4,11227634,JB147599,08/26/2017 10:00:00 AM,001XX W RANDOLPH ST,0281,CRIM SEXUAL ASSAULT,NON-AGGRAVATED,HOTEL/MOTEL,False,False,122,1.0,42.0,32.0,02,NaN,NaN,2017,02/11/2018 03:57:41 PM,NaN,NaN,NaN,1,0,0,0


In [7]:
crimes = data.copy()
crimes['Date'] = pd.to_datetime(crimes.Date, format='%m/%d/%Y %I:%M:%S %p')
# crimes.index = pd.DatetimeIndex(crimes.Date1)

crimes=crimes.sort_values(by='Date', ascending=True)
crimes.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Felony,Vagueness,Misdemeanor,HOMICIDE
1784561,1651806,G437744,2001-01-01,031XX N SEMINARY AV,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,RESIDENCE,False,False,1932,19.0,NaN,NaN,06,1168442.0,1920901.0,2001,08/17/2015 03:03:40 PM,41.938485,-87.656344,"(41.938484782, -87.656343697)",0,0,1,0
1551623,1353509,G057402,2001-01-01,034XX W ADAMS ST,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,RESIDENCE,False,False,1123,11.0,NaN,NaN,11,1153613.0,1898824.0,2001,08/17/2015 03:03:40 PM,41.878212,-87.711432,"(41.87821192, -87.711432075)",0,0,1,0
3132857,3361926,HK408994,2001-01-01,059XX N CLARK ST,1754,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,False,False,2012,20.0,40.0,77.0,02,1164565.0,1939287.0,2001,03/31/2006 10:03:38 PM,41.989020,-87.670069,"(41.989019862, -87.67006944)",0,1,0,0
3357765,3661792,HK760148,2001-01-01,054XX S DREXEL AVE,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,2131,2.0,5.0,41.0,06,1183119.0,1869343.0,2001,03/31/2006 10:03:38 PM,41.796676,-87.604013,"(41.796676399, -87.604012947)",0,0,1,0
1564351,1370284,G074254,2001-01-01,003XX N MASON AV,0820,THEFT,$500 AND UNDER,PARKING LOT/GARAGE(NON.RESID.),False,False,1512,15.0,NaN,NaN,06,1136750.0,1901645.0,2001,08/17/2015 03:03:40 PM,41.886272,-87.773282,"(41.886271732, -87.773282434)",0,0,1,0


In [8]:
crimes['year']=crimes.Date.dt.year
crimes['month']=crimes.Date.dt.month

In [9]:
crimes['crime_type'] = crimes['Misdemeanor']
crimes.loc[crimes['Felony']==1 ,'crime_type'] = 2
crimes.loc[crimes['HOMICIDE']==1 ,'crime_type'] = 3
# crimes.head()

In [10]:
crimes['Mis_sum'] = ((crimes['Arrest']==False) & (crimes['crime_type']==1)).cumsum()
crimes['Fel_sum'] = ((crimes['Arrest']==False) & (crimes['crime_type']==2)).cumsum()
crimes['Hom_sum'] = ((crimes['Arrest']==False) & (crimes['crime_type']==3)).cumsum()

crimes['Fel/Mis'] = (crimes['Fel_sum']/crimes['Mis_sum'])
crimes['Hom/Fel'] = (crimes['Hom_sum']/crimes['Fel_sum'])

# crimes['year'] = crimes['year'].astype('string')
# crimes['month'] = crimes['month'].astype('string')

In [11]:
crimes.head()

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Felony,Vagueness,Misdemeanor,HOMICIDE,year,month,crime_type,Mis_sum,Fel_sum,Hom_sum,Fel/Mis,Hom/Fel
1784561,1651806,G437744,2001-01-01,031XX N SEMINARY AV,0841,THEFT,FINANCIAL ID THEFT:$300 &UNDER,RESIDENCE,False,False,1932,19.0,NaN,NaN,06,1168442.0,1920901.0,2001,08/17/2015 03:03:40 PM,41.938485,-87.656344,"(41.938484782, -87.656343697)",0,0,1,0,2001,1,1,1,0,0,0.0,NaN
1551623,1353509,G057402,2001-01-01,034XX W ADAMS ST,1150,DECEPTIVE PRACTICE,CREDIT CARD FRAUD,RESIDENCE,False,False,1123,11.0,NaN,NaN,11,1153613.0,1898824.0,2001,08/17/2015 03:03:40 PM,41.878212,-87.711432,"(41.87821192, -87.711432075)",0,0,1,0,2001,1,1,2,0,0,0.0,NaN
3132857,3361926,HK408994,2001-01-01,059XX N CLARK ST,1754,OFFENSE INVOLVING CHILDREN,AGG SEX ASSLT OF CHILD FAM MBR,RESIDENCE,False,False,2012,20.0,40.0,77.0,02,1164565.0,1939287.0,2001,03/31/2006 10:03:38 PM,41.989020,-87.670069,"(41.989019862, -87.67006944)",0,1,0,0,2001,1,0,2,0,0,0.0,NaN
3357765,3661792,HK760148,2001-01-01,054XX S DREXEL AVE,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,2131,2.0,5.0,41.0,06,1183119.0,1869343.0,2001,03/31/2006 10:03:38 PM,41.796676,-87.604013,"(41.796676399, -87.604012947)",0,0,1,0,2001,1,1,3,0,0,0.0,NaN
1564351,1370284,G074254,2001-01-01,003XX N MASON AV,0820,THEFT,$500 AND UNDER,PARKING LOT/GARAGE(NON.RESID.),False,False,1512,15.0,NaN,NaN,06,1136750.0,1901645.0,2001,08/17/2015 03:03:40 PM,41.886272,-87.773282,"(41.886271732, -87.773282434)",0,0,1,0,2001,1,1,4,0,0,0.0,NaN


In [25]:
df3 = crimes[['Date','Arrest', 'Ward', 'Latitude', 'Longitude',
       'Felony', 'Vagueness', 'Misdemeanor', 'HOMICIDE', 'year','month', 'crime_type']]
df3.reset_index(inplace=True)
df3.head()

,index,Date,Arrest,Ward,Latitude,Longitude,Felony,Vagueness,Misdemeanor,HOMICIDE,year,month,crime_type
0,1784561,2001-01-01 00:00:00,False,NaN,41.938485,-87.656344,0,0,1,0,2001,1,1
1,1551623,2001-01-01 00:00:00,False,NaN,41.878212,-87.711432,0,0,1,0,2001,1,1
2,3132857,2001-01-01 00:00:00,False,40.0,41.989020,-87.670069,0,1,0,0,2001,1,0
3,3357765,2001-01-01 00:00:00,False,5.0,41.796676,-87.604013,0,0,1,0,2001,1,1
4,1564351,2001-01-01 00:00:00,False,NaN,41.886272,-87.773282,0,0,1,0,2001,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7227295,418750,2020-11-04 23:45:00,False,3.0,NaN,NaN,0,0,1,0,2020,11,1
7227296,418857,2020-11-04 23:49:00,True,6.0,NaN,NaN,0,0,1,0,2020,11,1
7227297,419023,2020-11-04 23:51:00,False,28.0,NaN,NaN,0,0,1,0,2020,11,1
7227298,419179,2020-11-04 23:56:00,True,4.0,NaN,NaN,0,0,1,0,2020,11,1


In [26]:
df3['Fel_w'] = pd.DataFrame(df3['Felony'].groupby(df3['Ward']).cumsum())
df3['Mis_w'] = pd.DataFrame(df3['Misdemeanor'].groupby(df3['Ward']).cumsum())
df3['Hom_w'] = pd.DataFrame(df3['HOMICIDE'].groupby(df3['Ward']).cumsum())

In [27]:
df3 = df3.dropna(axis=0)
df3.reset_index(inplace=True)

In [28]:
df3.columns

Index(['level_0', 'index', 'Date', 'Arrest', 'Ward', 'Latitude', 'Longitude',
       'Felony', 'Vagueness', 'Misdemeanor', 'HOMICIDE', 'year', 'month',
       'crime_type', 'Fel_w', 'Mis_w', 'Hom_w'],
      dtype='object')

In [29]:
df3 = pd.DataFrame(df3.groupby(['year', 'month', 'Ward', 'crime_type'])['Date', 'Arrest', 'Ward', 'Latitude', 'Longitude', 'crime_type', 'Fel_w', 'Mis_w', 'Hom_w'].head(1))
df3.reset_index(inplace=True)

In [34]:
df3.head()

,index,Date,Arrest,Ward,Latitude,Longitude,crime_type,Fel_w,Mis_w,Hom_w
0,0,2001-01-01 00:00:00,False,40.0,41.989020,-87.670069,0,0,0,0
1,1,2001-01-01 00:00:00,False,5.0,41.796676,-87.604013,1,0,1,0
2,2,2001-01-01 00:00:00,False,17.0,41.763415,-87.629872,1,0,1,0
3,3,2001-01-01 00:00:00,False,7.0,41.746697,-87.549015,2,1,0,0
4,4,2001-01-01 00:00:00,False,43.0,41.919153,-87.636402,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...
40454,6538879,2020-10-29 14:41:00,True,46.0,41.956711,-87.652882,3,16721,67975,77
40455,6538886,2020-11-02 20:36:00,False,28.0,41.876581,-87.745138,3,103710,185366,694
40456,6538887,2020-11-03 13:33:00,False,24.0,41.870829,-87.716346,3,85487,162263,624
40457,6538889,2020-11-04 07:00:00,False,29.0,41.880360,-87.755936,3,38933,105164,305


In [40]:
sample = df3.pivot_table(values='Mis_w', index='Ward', columns=['Date'])
sample.to_csv('/content/drive/My Drive/sample_file_mis.tsv')

In [41]:
sample = df3.pivot_table(values='Fel_w', index='Ward', columns=['Date'])
sample.to_csv('/content/drive/My Drive/sample_file_Fel.tsv')

In [42]:
sample = df3.pivot_table(values='Hom_w', index='Ward', columns=['Date'])
sample.to_csv('/content/drive/My Drive/sample_file_hom.tsv')